# Reading images

Astropy provides a few ways to read in FITS images, some in the core package and others in affiliated packages.

Before exploring those, we'll create a set of (fake) images to work with.

In [1]:
from pathlib import Path
from astropy.nddata import CCDData
from astropy.io import fits

## Working with diretories

The cell below contains the path to the images. In this notebook we'll use that both to store the fake images we generate and to read images. In normal use you wouldn't start by writing images there, of course. 

If the images are in the same directory as the notebook you can omit this, or set it to an empty string `''`. That case -- images in the same directory as the notebook -- is straightforward, but it is not at all uncommon to need to work with images in a different directory.

Later we'll look at how to generate the full path to an image (directory plus file name) in a way that will work on any platform. One of the approaches to loading images (using `ccdproc.ImageFileCollection`) lets you foprget about this, mostly.

In [2]:
data_directory = 'path/to/my/images'

## Generate some fake images


In [3]:
from pathlib import Path
from itertools import cycle

import numpy as np

image_path = Path(data_directory)

image_path.mkdir(parents=True, exist_ok=True)

images_to_generate = {
    'BIAS': 5,
    'DARK': 10,
    'FLAT': 3,
    'LIGHT': 10
}

exposure_times = {
    'BIAS': [0.0],
    'DARK': [5.0, 30.0],
    'FLAT': [5.0, 6.1, 7.3],
    'LIGHT': [30.0],
}

filters = {
    'FLAT': 'V',
    'LIGHT': 'V'
}

objects = {
    'LIGHT': ['m82', 'xx cyg']
}

image_size = [300, 200]

image_number = 0
for image_type, num in images_to_generate.items():
    exposures = cycle(exposure_times[image_type])
    try:
        filts = cycle(filters[image_type])
    except KeyError:
        filts = []
    
    try:
        objs = cycle(objects[image_type])
    except KeyError:
        objs = []
    for _ in range(num):
        img = CCDData(data=np.random.randn(*image_size), unit='adu')
        img.meta['IMAGETYP'] = image_type
        img.meta['EXPOSURE'] = next(exposures)
        if filts:
            img.meta['FILTER'] = next(filts)
        if objs:
            img.meta['OBJECT'] = next(objs)
        image_name = str(image_path / f'img-{image_number:04d}.fits')
        img.write(image_name)
        print(image_name)
        image_number += 1

path/to/my/images/img-0000.fits
path/to/my/images/img-0001.fits
path/to/my/images/img-0002.fits
path/to/my/images/img-0003.fits
path/to/my/images/img-0004.fits
path/to/my/images/img-0005.fits
path/to/my/images/img-0006.fits
path/to/my/images/img-0007.fits
path/to/my/images/img-0008.fits
path/to/my/images/img-0009.fits
path/to/my/images/img-0010.fits
path/to/my/images/img-0011.fits
path/to/my/images/img-0012.fits
path/to/my/images/img-0013.fits
path/to/my/images/img-0014.fits
path/to/my/images/img-0015.fits
path/to/my/images/img-0016.fits
path/to/my/images/img-0017.fits
path/to/my/images/img-0018.fits
path/to/my/images/img-0019.fits
path/to/my/images/img-0020.fits
path/to/my/images/img-0021.fits
path/to/my/images/img-0022.fits
path/to/my/images/img-0023.fits
path/to/my/images/img-0024.fits
path/to/my/images/img-0025.fits
path/to/my/images/img-0026.fits
path/to/my/images/img-0027.fits


## Option 1: Reading a single image with `astropy.io.fits`

This option gives you the most flexibility but is the least adapted to CCD images specifically. What you read in is a list of FITS extensions; you must select the one you want then access the data or header as desired. 

We'll open up the first of the fake images, `img-0001.fits`. To combine that with the directory name we'll use Python 3's `pathlib`; it ensures that the path combination will work on Windows too.

In [3]:
image_name = 'img-0001.fits'

image_path = Path(data_directory) / image_name

hdu_list = fits.open(image_path)
hdu_list

The `hdu_list` is a list of FITS Header-Data Units; in this case there is just one, containing both the image header and data, accessed as shown below.

In [4]:
hdu = hdu_list[0]
hdu.header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  200                                                  
NAXIS2  =                  300                                                  
IMAGETYP= 'BIAS    '                                                            
EXPOSURE=                  0.0                                                  
BUNIT   = 'adu     '                                                            

In [5]:
hdu.data

array([[ 0.91962546, -0.94398497,  0.24254976, ..., -1.33550249,
        -0.92099482,  1.73339116],
       [-0.26848847,  0.64501649,  0.09068821, ...,  1.64452457,
         0.94857587,  0.55936338],
       [-0.35847435,  0.11360779, -0.50357318, ...,  0.40934905,
        -1.78763228,  1.06970558],
       ...,
       [-0.25411335,  0.05358174,  1.50695881, ..., -1.60197614,
        -0.628959  , -0.23650373],
       [ 1.14317143,  0.95321118,  0.3985287 , ..., -0.66414859,
        -0.63215069, -0.53788723],
       [-1.1468373 , -0.39698568, -0.15371635, ..., -1.78283199,
        -1.33742184, -1.00997915]])

## Option 2:  Use `CCDData` to read in a single image

Astropy contains a `CCDData` object for representing a single image. It is not as flexible as using `astrop.io.fits` directly (for example, it assumes there is only one FITS extension and that it contains image data) but it sets up several properties that make the data easier to work with. 

We'll read in the same single image we did in the example above, `img-0001.fits`

In [6]:
ccd = CCDData.read(image_path)

The data and header are accssed in a way similar to the way you access it in an HDU returned by `astropy.io.fits`:

In [7]:
ccd.header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  200                                                  
NAXIS2  =                  300                                                  
IMAGETYP= 'BIAS    '                                                            
EXPOSURE=                  0.0                                                  
BUNIT   = 'adu     '                                                            

In [8]:
ccd.data

array([[ 0.91962546, -0.94398497,  0.24254976, ..., -1.33550249,
        -0.92099482,  1.73339116],
       [-0.26848847,  0.64501649,  0.09068821, ...,  1.64452457,
         0.94857587,  0.55936338],
       [-0.35847435,  0.11360779, -0.50357318, ...,  0.40934905,
        -1.78763228,  1.06970558],
       ...,
       [-0.25411335,  0.05358174,  1.50695881, ..., -1.60197614,
        -0.628959  , -0.23650373],
       [ 1.14317143,  0.95321118,  0.3985287 , ..., -0.66414859,
        -0.63215069, -0.53788723],
       [-1.1468373 , -0.39698568, -0.15371635, ..., -1.78283199,
        -1.33742184, -1.00997915]])

## Option 3: Working with a directory of images using `ImageFileCollection`

The affiliated package [cdcproc](https://ccdproc.readthedocs.io/) provides an easier way to work with collections of images in a directory: an `ImageFileCollection`. The `ImageFileCollection` is initialized with the name of the directory containing the images.

In [9]:
from ccdproc import ImageFileCollection
im_collection = ImageFileCollection(data_directory)

Note that we didn't need to worry about using `pathlib` to combine the directory and file name, we just give the collection the name of the directory.

### Summary of directory contents

The `summary` property provides an overview of the files in the directory; it is an astropy `Table`, so you can access columns in the usual way.

In [10]:
im_collection.summary

file,simple,bitpix,naxis,naxis1,naxis2,imagetyp,exposure,bunit,filter,object
str13,bool,int64,int64,int64,int64,str5,float64,str3,object,object
img-0000.fits,True,-64,2,200,300,BIAS,0.0,adu,--,--
img-0001.fits,True,-64,2,200,300,BIAS,0.0,adu,--,--
img-0002.fits,True,-64,2,200,300,BIAS,0.0,adu,--,--
img-0003.fits,True,-64,2,200,300,BIAS,0.0,adu,--,--
img-0004.fits,True,-64,2,200,300,BIAS,0.0,adu,--,--
img-0005.fits,True,-64,2,200,300,DARK,5.0,adu,--,--
img-0006.fits,True,-64,2,200,300,DARK,30.0,adu,--,--
img-0007.fits,True,-64,2,200,300,DARK,5.0,adu,--,--
img-0008.fits,True,-64,2,200,300,DARK,30.0,adu,--,--


### Filtering and iterating over images

The convenient thing about `ImageFileCollection` is that it provides easy ways to filter or loop over files via FITS header keyword values. 

For example, looping over just the flat files is just one line of code:

In [11]:
for a_flat in im_collection.hdus(imagetyp='FLAT'):
    print(a_flat.header['EXPOSURE'])

5.0
6.1
7.3


Instead of iterating over HDUs, as in the example above, you can iterate over just the headers (with `.headers`) or just the data (with `.data`). You can use any FITS keyword from the header as a keyword for selecting the images you want. In addition, you can return the file name while iterating also.

In [12]:
for a_flat, fname in im_collection.hdus(imagetyp='LIGHT', object='m82', return_fname=True):
    print(f'In file {fname} the exposure is:', a_flat.header['EXPOSURE'], 'with standard deviation ', a_flat.data.std())

In file img-0018.fits the exposure is: 30.0 with standard deviation  1.0022074490439763
In file img-0020.fits the exposure is: 30.0 with standard deviation  0.9991078620421082
In file img-0022.fits the exposure is: 30.0 with standard deviation  1.0023580633672473
In file img-0024.fits the exposure is: 30.0 with standard deviation  1.000477776445242
In file img-0026.fits the exposure is: 30.0 with standard deviation  0.9993075844461872


The [documentation for `ImageFileCollection`](https://ccdproc.readthedocs.io/en/latest/ccdproc/image_management.html) describes more of its capabilities; it can also automatically save a copy of each image as you iterate over them, for example.

In [24]:
for a_flat, fname in im_collection.ccds(bunit='ADU', return_fname=True):
    print(a_flat.unit)

adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu


In [16]:
a_flat.header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  200                                                  
NAXIS2  =                  300                                                  
IMAGETYP= 'LIGHT   '                                                            
EXPOSURE=                 30.0                                                  
FILTER  = 'V       '                                                            
OBJECT  = 'm82     '                                                            
BUNIT   = 'adu     '                                                            